In [6]:
! pip install SpeechRecognition pydub langdetect googletrans==4.0.0-rc1 pyaudio

  Using cached PyAudio-0.2.14-cp312-cp312-win_amd64.whl.metadata (2.7 kB)
Using cached PyAudio-0.2.14-cp312-cp312-win_amd64.whl (164 kB)


In [10]:
import speech_recognition as sr
from googletrans import Translator

def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio = recognizer.listen(source, timeout=5)
    try:
        # Try recognizing using English (India)
        text = recognizer.recognize_google(audio, language="en-IN")
        print("You said (EN):", text)
        return text
    except:
        try:
            # Try recognizing using Hindi
            text = recognizer.recognize_google(audio, language="hi-IN")
            print("You said (HI):", text)
            return text
        except Exception as e:
            print("Error:", e)
            return None

def detect_language(text):
    translator = Translator()
    try:
        detected = translator.detect(text)
        if detected.lang == 'hi':
            # Simple check for Hinglish elements
            if any(word in text.lower() for word in ["is", "are", "you", "meeting", "hai", "ho", "the", "my","main","hoon"]):
                return "Hinglish"
            return "Hindi"
        elif detected.lang == 'en':
            return "English"
        else:
            return "Unknown"
    except:
        return "Error in detection"

def translate_to_english(text):
    translator = Translator()
    try:
        translated = translator.translate(text, dest='en')
        return translated.text
    except:
        return "Translation failed"
def translate_to_hindi(text):
    translator = Translator()
    try:
        translated = translator.translate(text, dest='hi')
        return translated.text
    except:
        return "Translation failed"

if __name__ == "__main__":
    while True:
        text = speech_to_text()
        if text:
            lang = detect_language(text)
            print(f"Detected Language: {lang}")

            if lang != "English":
                english_transcript = translate_to_english(text)
                print(f"English Translation: {english_transcript}")
            else:
                english_transcript = translate_to_english(text)
                print(f"English Translation: {english_transcript}")


Speak now...
You said (EN): mein kitne Dard Se Gujar rahi hun
Detected Language: Hindi
English Translation: How many pain i am going through
Speak now...
You said (EN): I just don't know yaar this is very complicated
Detected Language: English
English Translation: I just don't know yaar this is very complicated
Speak now...
You said (EN): baat samajhne ki koshish to kar main bhi Yahi bolna chah Rahi
Detected Language: Hinglish
English Translation: Trying to understand the matter, I want to speak the same
Speak now...
You said (EN): pata nahin kab samjhenge log
Detected Language: Hindi
English Translation: Don't know when people will understand
Speak now...
You said (EN): badhiya hai
Detected Language: Hinglish
English Translation: Great
Speak now...


WaitTimeoutError: listening timed out while waiting for phrase to start

In [13]:
import speech_recognition as sr
from googletrans import Translator
import time

def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("\nSpeak now...")
        try:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio = recognizer.listen(source, timeout=5)
            try:
                # Try English first
                text = recognizer.recognize_google(audio, language="en-IN")
                print("You said (EN):", text)
                return text
            except sr.UnknownValueError:
                # Fall back to Hindi
                text = recognizer.recognize_google(audio, language="hi-IN")
                print("You said (HI):", text)
                return text
        except Exception as e:
            print("Error in listening:", str(e))
            return None

def detect_language(text):
    if not text:
        return "Unknown"
    
    translator = Translator()
    try:
        detected = translator.detect(text)
        if detected.lang == 'hi':
            # Enhanced Hinglish detection
            hinglish_triggers = ["is", "are", "you", "hai", "ho", "the", "my", "main", "hoon", "doing"]
            if any(word in text.lower() for word in hinglish_triggers):
                return "Hinglish"
            return "Hindi"
        elif detected.lang == 'en':
            return "English"
        return "Unknown"
    except Exception as e:
        print("Detection error:", str(e))
        return "Unknown"

def translate_to_english(text):
    if not text:
        return "No text to translate"
    
    translator = Translator()
    try:
        translated = translator.translate(text, dest='en')
        return translated.text
    except Exception as e:
        print("Translation error:", str(e))
        return "Translation failed"

def main():
    max_attempts = 100000  # Prevent infinite loops
    attempt = 0
    
    while attempt < max_attempts:
        attempt += 1
        try:
            text = speech_to_text()
            if not text:
                time.sleep(1)
                continue
                
            lang = detect_language(text)
            print(f"Detected Language: {lang}")
            
            if lang == "Hindi":
                print(f"\nHindi: {text}")
                print(f"English: {translate_to_english(text)}")
            elif lang == "Hinglish":
                print(f"\nHinglish: {text}")
                print(f"English: {translate_to_english(text)}")
            elif lang == "English":
                print(f"\nEnglish: {text}")
                # No translation needed for English
            else:
                print("\nCould not detect language")
            
            print("\n" + "="*40)
            time.sleep(1)  # Add delay between attempts
            
        except KeyboardInterrupt:
            print("\nExiting...")
            break
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
            time.sleep(2)

if __name__ == "__main__":
    main()


Speak now...
You said (EN): you are very annoying
Detected Language: English

English: you are very annoying


Speak now...
You said (EN): not proud of you
Detected Language: English

English: not proud of you


Speak now...
You said (EN): to yah drama kab tak karegi
Detected Language: Hinglish

Hinglish: to yah drama kab tak karegi
English: So how long will this drama do


Exiting...
